Exploring a Textual Corpus with ArchiTXT
========================================

This tutorial provides a **step-by-step guide** on how to use **ArchiTXT** to efficiently process and analyze textual corpora.

ArchiTXT allows loading a corpus as a set of syntax trees, where each tree is enriched by incorporating named entities.
These enriched trees form a **forest**, which can then be automatically structured into a valid **database instance** for further analysis.

By following this tutorial, you'll learn how to:
- Load a corpus
- Parse textual data with **Berkeley Neural Parser (Benepar)**
- Extract structured data using **ArchiTXT**

In [ ]:
import itables

itables.init_notebook_mode(connected=True)

## Downloading the MACCROBAT Corpus
The **MACCROBAT** corpus is a collection of **200 annotated medical documents**, specifically **clinical case reports**, extracted from **PubMed Central**.
The annotations focus on key medical concepts such as **diseases, treatments, medications, and symptoms**, making it a valuable resource for biomedical text analysis.

The **MACCROBAT** corpus is available for download at [Figshare](https://figshare.com/articles/dataset/MACCROBAT2018/9764942).

Let's download the corpora.

In [ ]:
import io
import urllib.request
import zipfile

with urllib.request.urlopen('https://figshare.com/ndownloader/articles/9764942/versions/2') as response:
    archive_file = io.BytesIO(response.read())

with zipfile.ZipFile(archive_file) as archive:
    archive.extract('MACCROBAT2020.zip')

## Installing and Configuring NLP Models

ArchiTXT can parse the sentences using either **Benepar** with SpaCy or a **CoreNLP** server.
In this tutorial, we will use the **SpaCy parser** with the default model, but you can use any models like one from **SciSpaCy**, a collection of models designed for biomedical text processing by **AllenAI**.

To download the SciSpaCy model, do:

In [ ]:
!spacy download en_core_web_sm

We also need to download the Benepar model for English

In [ ]:
import benepar

benepar.download('benepar_en3')

## Parsing the Corpus with ArchiTXT

Before processing the corpus, we need to configure the **BeneparParser**, specifying which SpaCy model to use for each language.

In [ ]:
import warnings

from architxt.nlp.parser.benepar import BeneparParser

# Initialize the parser
parser = BeneparParser(
    spacy_models={
        'English': 'en_core_web_sm',
    }
)

# Suppress warnings for unsupported annotations
warnings.filterwarnings("ignore")

Named Entity Resolution (NER) helps to standardize the named entities and to build a database instance.
To enable NER, we need to provide the knowledge base to use.
For this tutorial, we will use the **UMLS (Unified Medical Language System)** resolver.

Let's parse a sample of the corpus. To verify that everything is functioning as expected, we will inspect the largest enriched tree using the :py:meth:`~architxt.tree.Tree.pretty_print` method.

In [ ]:
from architxt.nlp import raw_load_corpus

forest = [
    tree
    async for tree in raw_load_corpus(
        ['MACCROBAT2020.zip'],
        ['English'],
        parser=parser,
        resolver_name='umls',
        sample=400,
    )
]

# Look at the highest tree
max(forest, key=lambda tree: tree.height).pretty_print()

 **ArchiTXT** can then automatically structure parsed text into a **database-friendly format**.
 Let's start with a simple rewrite!

In [ ]:
from copy import deepcopy

from architxt.simplification.simple_rewrite import simple_rewrite

forest_copy = deepcopy(forest)
simple_rewrite(forest_copy)

In [ ]:
# Look at the highest tree
max(forest_copy, key=lambda tree: tree.height).pretty_print()

Now that we have a structured instance, we can extract its schema.
The schema provides a **formal representation** of the extracted data.

In [ ]:
from architxt.schema import Schema

schema = Schema.from_forest(forest_copy, keep_unlabelled=False)
print(schema.as_cfg())

We've successfully built a basic database schema from our corpus, but there's significant potential for improvement.
Let's explore how we can enhance it using the **ArchiTXT** simplification algorithm!

First, let's visualize the repartition of equivalent classes inside the forest.

In [ ]:
from architxt.similarity import equiv_cluster

clusters = equiv_cluster(forest, tau=0.8)

In [ ]:
import plotly.express as px

fig = px.bar(sum(tree.height == 2 for tree in klass) for klass in clusters)
fig.update_layout(xaxis_title='Equivalent Class', yaxis_title='Count', showlegend=False)

It's now time to use **ArchiTXT** to automatically structure the data.

In [ ]:
from architxt.simplification.tree_rewriting import rewrite

rewrite(forest, epoch=10, min_support=5, tau=0.8)

In [ ]:
# Look at the highest tree
max(forest, key=lambda tree: tree.height).pretty_print()

We now have a more granular structure. Let's take a closer look at the schema.

In [ ]:
schema = Schema.from_forest(forest, keep_unlabelled=False)
print(schema.as_cfg())

The schema is now much smaller, and the groups are more meaningful.

But not all extracted trees provide valuable insights, so we could filter the structured instance to keep only the **valid trees** using `schema.extract_valid_trees(new_forest)`.
Let's explore the different **semantic groups**.
Groups represent common patterns across the corpus.

In [ ]:
all_datasets = schema.extract_datasets(forest)
group, dataset = max(all_datasets.items(), key=lambda x: len(x[1]))

print(f'Group: {group.name}')

dataset